In [1]:
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions

In [2]:
classifier = VGG16()

In [3]:
classifier.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
suitImg = image.load_img("suit.jpg", target_size=(224, 224))

In [7]:
transformed_img = image.img_to_array(suitImg)
transformed_img = np.expand_dims(transformed_img, axis = 0)
transformed_img = preprocess_input(transformed_img)

In [8]:
pred = classifier.predict(transformed_img)
decode_predictions(pred, top = 5)
label = decode_predictions(pred)

40960/35363 [==================================] - 0s 0us/step


In [9]:
print('%s (%.2f%%)' % (label[0][0][1], label[0][0][2] * 100))

suit (82.98%)


In [10]:
print('%s (%.2f%%)' % (label[0][1][1], label[0][1][2] * 100))

Windsor_tie (6.13%)


In [11]:
print('%s (%.2f%%)' % (label[0][2][1], label[0][2][2] * 100))

trench_coat (2.20%)


In [12]:
for i in range(5):
    print('%16s (%.2f%%)' % (label[0][i][1], label[0][i][2] * 100))

            suit (82.98%)
     Windsor_tie (6.13%)
     trench_coat (2.20%)
           groom (1.88%)
            jean (1.31%)


In [13]:
test_img = image.load_img("pyramid.jpg", target_size = (224, 224))
test_img
trans_img = image.img_to_array(test_img)
trans_img = np.expand_dims(trans_img, axis = 0)
trans_img = preprocess_input(trans_img)
pred = classifier.predict(trans_img)

In [14]:
label = decode_predictions(pred)
for i in range(5):
    print('%16s (%.2f%%)' % (label[0][i][1], label[0][i][2] * 100))

            barn (28.18%)
          church (24.99%)
       bell_cote (13.44%)
         obelisk (7.30%)
          thatch (5.33%)


In [15]:
last_layer = str(classifier.layers[-1])

In [20]:
import tensorflow.keras
from tensorflow.keras.layers import Dense
new_layer = tensorflow.keras.Sequential()
for layer in classifier.layers:
    if str(layer) != last_layer:
        new_layer.add(layer)
new_layer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [22]:
for layer in new_layer.layers:
    layer.trainable = False
new_layer.add(
    Dense(128, activation = 'relu')
)
new_layer.add(
    Dense(64, activation = 'relu')
)
new_layer.add(
    Dense(32, activation = 'relu')
)
new_layer.add(
    Dense(16, activation = 'relu')
)
new_layer.add(
    Dense(1, activation = 'sigmoid')
)
new_layer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [26]:
new_layer.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
train_gen = ImageDataGenerator(rescale = 1. / 255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
test_gen = ImageDataGenerator(rescale = 1.0 / 255)

In [29]:
trainSet = train_gen.flow_from_directory('Applied-Deep-Learning-with-Keras/Lesson07/Datasets/dataset/training_set',target_size = (224, 224),batch_size = 32,class_mode = 'binary')
testSet = test_gen.flow_from_directory('Applied-Deep-Learning-with-Keras/Lesson07/Datasets/dataset/test_set',target_size = (224, 224),batch_size = 32,class_mode = 'binary')
batch_size = 100
new_layer.fit_generator(trainSet,steps_per_epoch = int(10000/batch_size),epochs = 10,validation_data = testSet,validation_steps = int(2500/batch_size))

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
100/100 [==============================] - 381s 4s/step - loss: 0.5519 - accuracy: 0.7025 - val_loss: 0.3840 - val_accuracy: 0.8213
Epoch 2/10
100/100 [==============================] - 387s 4s/step - loss: 0.4203 - accuracy: 0.8022 - val_loss: 0.3043 - val_accuracy: 0.8788
Epoch 3/10
100/100 [==============================] - 379s 4s/step - loss: 0.3160 - accuracy: 0.8644 - val_loss: 0.2830 - val_accuracy: 0.8625
Epoch 4/10
100/100 [==============================] - 368s 4s/step - loss: 0.3233 - accuracy: 0.8572 - val_loss: 0.3142 - val_accuracy: 0.8737
Epoch 5/10
 55/100 [===============>..............] - ETA: 2:11 - loss: 0.2906 - accuracy: 0.8687

D:\anaconda\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  warnings.warn(
D:\anaconda\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  warnings.warn(
D:\anaconda\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  warnings.warn(
D:\anaconda\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  warnings.warn(
D:\anaconda\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 283
  warnings.warn(
D:\anaconda\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2

100/100 [==============================] - 367s 4s/step - loss: 0.3231 - accuracy: 0.8544 - val_loss: 0.3417 - val_accuracy: 0.8325
Epoch 6/10
100/100 [==============================] - 371s 4s/step - loss: 0.3153 - accuracy: 0.8581 - val_loss: 0.2419 - val_accuracy: 0.9062
Epoch 7/10
100/100 [==============================] - 370s 4s/step - loss: 0.3221 - accuracy: 0.8581 - val_loss: 0.2501 - val_accuracy: 0.8975
Epoch 8/10
100/100 [==============================] - 372s 4s/step - loss: 0.2731 - accuracy: 0.8866 - val_loss: 0.3159 - val_accuracy: 0.8525
Epoch 9/10
100/100 [==============================] - 371s 4s/step - loss: 0.3076 - accuracy: 0.8625 - val_loss: 0.2625 - val_accuracy: 0.8750
Epoch 10/10
100/100 [==============================] - 370s 4s/step - loss: 0.2692 - accuracy: 0.8866 - val_loss: 0.2164 - val_accuracy: 0.9125


In [31]:
cat_img = image.load_img("test_image_2.jpg", target_size = (224, 224))
cat_img
trans_img = image.img_to_array(cat_img)
trans_img = np.expand_dims(trans_img, axis = 0)
trans_img = preprocess_input(trans_img)
res = new_layer.predict(trans_img)
if res[0][0] == 1:
    prediction = "Dog"
else:
    prediction = "Cat"
print(prediction)

Cat
